In [1]:
import pandas as pd
from openai import OpenAI
import ast
import json
import os
import dotenv
import psycopg2
from psycopg2 import sql

In [2]:

dotenv.load_dotenv("F:\\AI_agent_DA\\env.txt")

connection_dtl = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'dbname': os.getenv('DB_NAME')
}

def db_connect(connection_dtl):
    
    connection = psycopg2.connect(
    host=connection_dtl['host'],
    port=connection_dtl['port'],
    user=connection_dtl['user'],
    password=connection_dtl['password'],
    dbname=connection_dtl['dbname']
    )
    return connection

def execute_sql_query(connection_dtl, query):
    connection = db_connect(connection_dtl=connection_dtl)
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cols = [cursor.description[col][0] for col in range(len(cursor.description))]
        connection.close()
        return pd.DataFrame(result, columns=cols)
    except Exception as e:
        print("SQL Error")
        return pd.DataFrame()
        
llm_model = "gpt-4o-mini"
#llm_model = "gpt-4"

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
client = OpenAI()

def get_completion(prompt, model=llm_model):
    
    response = client.chat.completions.create(
    model=llm_model,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )

    return response.choices[0].message.content


In [ ]:
# tables and instructions functions

def fetch_table(table_name):
    query = f"""
    SELECT * 
    FROM {table_name}
    """
    return execute_sql_query(connection_dtl, query)



def analyse_with_ai(df):
    prompt = f"""
        *** Respond strictly with same JSON structure,***
        you are data analyst and your task is find average of total_price for all payment_method in output dataframe there should be columns like payment_method,average_of_total_price (it is a float8 column which contain numbers)  using df: ```'{df}'```;"""
    response = get_completion(prompt,llm_model)

    # result_df['output'] = response_json


    return response



In [4]:
table_name = 'orders'
tbl = fetch_table(table_name)
tbl

,order_id,user_id,date,total_price,payment_method,shipping_address,zip_code
0,ORDER_1,CUSTOMER_4235,2021-02-08,593.0,Credit Card,"053, Brahma Samudram, Ananthapur, ANDHRA PRADESH",112448
1,ORDER_10,CUSTOMER_6,2023-07-03,4545.0,Credit Card,"628, Gushkara, Bardhaman, WEST BENGAL",112519
2,ORDER_100,CUSTOMER_1171,2022-09-30,2397.0,Cash on Delivery,"892, Paramthivelur, Namakkal, TAMIL NADU",110835
3,ORDER_1000,CUSTOMER_2777,2023-09-16,71384.0,Credit Card,"399, Gohad, Bhind, MADHYA PRADESH",112812
4,ORDER_10000,CUSTOMER_1774,2021-05-07,2791.0,Credit Card,"1237, Sadar, Gorakhpur, UTTAR PRADESH",110048
...,...,...,...,...,...,...,...
44982,ORDER_9995,CUSTOMER_3948,2022-10-26,55283.0,Debit Card,"118, Ganapavaram Mandal, West Godavari, ANDHRA...",111481
44983,ORDER_9996,CUSTOMER_1209,2021-07-24,3349.0,Wallet,"0542, Ambabhona, Bargarh, ODISHA",111937
44984,ORDER_9997,CUSTOMER_929,2023-03-01,793.0,Netbanking,"623, Chhotaudepur, Vadodara, GUJARAT",111369
44985,ORDER_9998,CUSTOMER_1565,2022-04-06,2961.0,Wallet,"1703, Mohammdabad Gohana, Mau, UTTAR PRADESH",112309


In [5]:
opt = analyse_with_ai(tbl)
opt

'```json\n{\n    "output": [\n        {\n            "payment_method": "Credit Card",\n            "average_of_total_price": 15144.3333\n        },\n        {\n            "payment_method": "Cash on Delivery",\n            "average_of_total_price": 2397.0\n        },\n        {\n            "payment_method": "Debit Card",\n            "average_of_total_price": 55283.0\n        },\n        {\n            "payment_method": "Wallet",\n            "average_of_total_price": 2655.0\n        },\n        {\n            "payment_method": "Netbanking",\n            "average_of_total_price": 4036.5\n        }\n    ]\n}\n```'

In [6]:
def convert_in_df(jsn):
    response_json = json.loads(jsn.strip("```json\n").strip())
    
    # Extract the 'result' list from the response
    result = response_json.get('output', [])
    
    # Convert the 'result' list into a DataFrame
    result_df = pd.DataFrame(result)
    
    return result_df

In [7]:
df = convert_in_df(opt)
df

,payment_method,average_of_total_price
0,Credit Card,15144.3333
1,Cash on Delivery,2397.0000
2,Debit Card,55283.0000
3,Wallet,2655.0000
4,Netbanking,4036.5000
